In [2]:
import numpy as np # type: ignore
import tensorflow as tf # type: ignore
import os
from tensorflow.keras.models import Model # type: ignore
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense, Reshape, Conv2DTranspose, Add, LeakyReLU, UpSampling2D, Dropout # type: ignore
from tensorflow.keras.optimizers import SGD, Adam # type: ignore
from tensorflow.keras.regularizers import l2 # type: ignore
from tensorflow.keras.optimizers.schedules import ExponentialDecay # type: ignore
from tensorflow.keras.losses import MeanSquaredError, CategoricalCrossentropy # type: ignore
from tensorflow.keras.metrics import MeanAbsoluteError, CategoricalAccuracy # type: ignore
from tensorflow.keras.preprocessing.image import ImageDataGenerator # type: ignore
from tensorflow.keras.utils import to_categorical # type: ignore
import keras_tuner as kt # type: ignore
import matplotlib.pyplot as plt

In [22]:
# Fonction qui genere des vecteurs aleatoires
def generate_unique_vectors(num_vectors, vector_length, vectors):
    vectors_list = []
    while len(vectors_list) < num_vectors:
        vector = tuple(np.random.randint(0, 2, vector_length))
        if vector not in vectors:
            vectors.add(vector)
            vectors_list.append(vector)
    return vectors_list, vectors

# Fonction qui convertie le generateur en tableau numpy
def generator_to_array(generator, class_vectors):
    samples = []
    vectors = []
    data_filenames = generator.filenames
    total_images = len(data_filenames)

    for i in range(len(generator)):
        batch = generator.next()
        batch_size = len(batch[0])

        for j in range(batch_size):
            index = i * generator.batch_size + j
            if index >= total_images:
                break  # Prevent going out of bounds

            samples.append(batch[0][j])
            class_name = data_filenames[index].split(os.path.sep)[0]
            vectors.append(class_vectors[class_name])
            
    return np.array(samples), np.array(vectors), data_filenames

# Fonction qui associe les images aux labels
def preprocess(train_generator, val_generator, num_classes, vector_length, total_vectors, use_random_vectors):
    class_indices = train_generator.class_indices
    
    if use_random_vectors:
        unique_vectors, total_vectors = generate_unique_vectors(num_classes, vector_length, total_vectors)
        class_vectors = {class_name: vector for class_name, vector in zip(class_indices, unique_vectors)}
    else:
        class_vectors = {class_name: i for i, class_name in enumerate(class_indices)}
    
    samples_train, vectors_train, _ = generator_to_array(train_generator, class_vectors)
    samples_val, vectors_val, _ = generator_to_array(val_generator, class_vectors)
    
    if not use_random_vectors:
        # Convert class indices to one-hot encoding
        vectors_train = to_categorical(vectors_train, num_classes=num_classes)
        vectors_val = to_categorical(vectors_val, num_classes=num_classes)
    
    return samples_train, vectors_train, samples_val, vectors_val, total_vectors

# Fonction qui charge les donnees
def load_data(datagen, target_size=(150, 150), batch_size=112, class_mode='input', shuffle=False, color_mode='grayscale', use_random_vectors=True, vector_length=56):
    ethnies = {'caucasians': [], 'afro_americans': [], 'asians': []}
    total_vectors = set()
    check = True
    for ethnie in ethnies.keys():
        trainset = datagen.flow_from_directory(f'../../Datasets/VGG/{ethnie}', target_size=target_size, batch_size=batch_size, class_mode=class_mode, shuffle=shuffle, color_mode=color_mode, subset='training')
        testset  = datagen.flow_from_directory(f'../../Datasets/VGG/{ethnie}', target_size=target_size, batch_size=batch_size, class_mode=class_mode, shuffle=shuffle, color_mode=color_mode, subset='validation')
        
        # Number of classes for one-hot encoding
        num_classes = len(trainset.class_indices)
        
        samples_train, vectors_train, samples_val, vectors_val, total_vectors = preprocess(trainset, testset, num_classes, vector_length, total_vectors, use_random_vectors)
        
        if trainset.n != samples_train.shape[0] or testset.n != samples_val.shape[0]:
            check = False
        ethnies[ethnie] = [trainset, testset, samples_train, vectors_train, samples_val, vectors_val]
    print(check)
    return ethnies

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.02, dtype='float16')

# Exemple vecteurs aleatoires
#ethnies_random = load_data(datagen, use_random_vectors=True)

# Exemple vecteur one-hot
#ethnies_onehot = load_data(datagen, use_random_vectors=False)
ethnies = load_data(datagen, target_size=(256, 256), use_random_vectors=False)

Found 4109 images belonging to 21 classes.
Found 75 images belonging to 21 classes.
Found 4363 images belonging to 22 classes.
Found 80 images belonging to 22 classes.
Found 4293 images belonging to 21 classes.
Found 75 images belonging to 21 classes.
True


In [4]:
#Définition des blocs résiudels
def residual_block(x, filters, kernel_size=3, stride=1):
    shortcut = x
    x = Conv2D(filters, kernel_size, strides=stride, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = Conv2D(filters, kernel_size, strides=1, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)

    if stride != 1 or shortcut.shape[-1] != filters:
        shortcut = Conv2D(filters, 1, strides=stride, padding='same', use_bias=False)(shortcut)
        shortcut = BatchNormalization()(shortcut)

    x = Add()([x, shortcut])
    x = LeakyReLU(alpha=0.1)(x)
    return x

def residual_block_recon(x, filters, kernel_size=3, stride=1):
    shortcut = x
    x = Conv2DTranspose(filters, kernel_size, strides=stride, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = Conv2DTranspose(filters, kernel_size, strides=1, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)

    shortcut = Conv2DTranspose(filters, 1, strides=stride, padding='same', use_bias=False)(shortcut)
    shortcut = BatchNormalization()(shortcut)

    x = Add()([x, shortcut])
    x = LeakyReLU(alpha=0.1)(x)
    return x

In [ ]:
def create_hybride(shape=(256, 256, 1), input_latent=512, len_vecteur=56):
    input_img = Input(shape=shape, name='input_image')
    #print(input_img.shape)
    x = Conv2D(16, 5, strides=2, padding='same', use_bias=False)(input_img)
    #print(x.shape)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(2, strides=2, padding='same')(x)
    #print(x.shape, "max pool")
    #encoder
    x = residual_block(x, 32, 3, 2)
    #print(x.shape)
    x = residual_block(x, 64, 3, 2)
    #print(x.shape)
    x = residual_block(x, 128, 3, 2)
    #print(x.shape)
    x = residual_block(x, 256, 3, 2)
    #print(x.shape)
    x = residual_block(x, 512, 3, 2)
    #print(x.shape, "bloc 512")
    x = Conv2D(input_latent, 3, strides=2, padding='same', use_bias=False)(x)
    #print(x.shape, "espace latent")

    #espace latent
    latent_space_layer = Dense(input_latent, activation='relu', use_bias=False)(x)
    latent_space_layer_norm = BatchNormalization(name='latent_space_layer_norm')(latent_space_layer)

    #decoder
    reshape_layer = Reshape(target_shape=(1, 1, input_latent))(latent_space_layer_norm)
    #print(reshape_layer.shape, 'reshape_layer')
    x_recon = residual_block_recon(reshape_layer, input_latent, 3, 2)
    #print(x_recon.shape, "bloc 512")
    x_recon = residual_block_recon(x_recon, 256, 3, 2)
    #print(x_recon.shape)
    x_recon = residual_block_recon(x_recon, 128, 3, 2)
    #print(x_recon.shape)
    x_recon = residual_block_recon(x_recon, 64, 3, 2)
    #print(x_recon.shape)
    x_recon = residual_block_recon(x_recon, 32, 3, 2)
    #print(x_recon.shape)
    x_recon = Conv2DTranspose(16, 2, strides=2, padding='same', use_bias=False)(x_recon)
    #print(x_recon.shape, "unverse max pool")
    x_recon = Conv2DTranspose(16, 5, strides=2, padding='same', use_bias=False)(x_recon)
    #print(x_recon.shape)
    x_recon = Conv2DTranspose(1, 1, activation='sigmoid', padding='same', use_bias=False)(x_recon)
    #print(x_recon.shape, 'recon_image')
    x_recon = tf.keras.layers.Resizing(height=256, width=256, name='recon_image')(x_recon)

    #classification
    x_class = Dense(128, activation='relu')(latent_space_layer_norm)
    x_class = Dense(len_vecteur, activation='softmax', name='classif')(x_class)

    model = Model(inputs=input_img, outputs=[x_recon, x_class])

    optimizer = tf.keras.optimizers.Adam(learning_rate=ExponentialDecay(initial_learning_rate=0.001, decay_steps=42237,decay_rate=0.5,staircase=False))
    model.compile(optimizer=optimizer, 
                  loss={'recon_image': MeanSquaredError(), 'classif': MeanSquaredError()},
                  metrics={'recon_image': MeanAbsoluteError(), 'classif': 'accuracy'})
    return model

In [ ]:
model_hybride = create_hybride()
model_hybride.summary()

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense, Reshape, Conv2DTranspose
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import MeanAbsoluteError
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.layers import LeakyReLU

def residual_block(x, filters, kernel_size, stride):
    shortcut = Conv2D(filters, 1, strides=stride, padding='same', use_bias=False)(x)
    shortcut = BatchNormalization()(shortcut)

    x = Conv2D(filters, kernel_size, strides=stride, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters, kernel_size, strides=1, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)

    x = tf.keras.layers.add([x, shortcut])
    x = Activation('relu')(x)
    return x

def residual_block_recon(x, filters, kernel_size, stride):
    shortcut = Conv2DTranspose(filters, 1, strides=stride, padding='same', use_bias=False)(x)
    shortcut = BatchNormalization()(shortcut)

    x = Conv2DTranspose(filters, kernel_size, strides=stride, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2DTranspose(filters, kernel_size, strides=1, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)

    x = tf.keras.layers.add([x, shortcut])
    x = Activation('relu')(x)
    return x

def create_autoencoder(shape=(256, 256, 1), input_latent=512):
    input_img = Input(shape=shape, name='input_image')
    print(input_img.shape)

    # Encoder
    x = Conv2D(16, 5, strides=2, padding='same', use_bias=False)(input_img)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    print(x.shape, "1ere conv")
    x = MaxPooling2D(2, strides=2, padding='same')(x)
    print(x.shape, "max pool")
    x = residual_block(x, 32, 3, 2)
    print(x.shape)
    x = residual_block(x, 64, 3, 2)
    print(x.shape)
    x = residual_block(x, 128, 3, 2)
    print(x.shape)
    x = residual_block(x, 256, 3, 2)
    print(x.shape)
    x = residual_block(x, 512, 3, 2)
    print(x.shape)
    x = residual_block(x, 1024, 3, 2)
    print(x.shape, "bloc 1024")
    x = Conv2D(input_latent, 3, strides=1, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation(LeakyReLU(alpha=0.1))(x)
    print(x.shape, "espace latent")
    x = Flatten()(x)
    
    # Latent space
    latent_space_layer = Dense(input_latent, activation='relu', use_bias=False)(x)
    latent_space_layer_norm = BatchNormalization(name='latent_space_layer_norm')(latent_space_layer)
    
    # Decoder
    reshape_layer = Reshape(target_shape=(1, 1, input_latent))(latent_space_layer_norm)
    print(reshape_layer.shape, 'reshape_layer')
    x_recon = Conv2DTranspose(input_latent, 3, strides=1, padding='same', use_bias=False)(reshape_layer)
    x_recon = BatchNormalization()(x_recon)
    x_recon = Activation(LeakyReLU(alpha=0.1))(x_recon)
    print(x_recon.shape)
    x_recon = residual_block_recon(x_recon, 1024, 3, 2)
    print(x_recon.shape, "bloc 1024")
    x_recon = residual_block_recon(x_recon, 512, 3, 2)
    print(x_recon.shape, "bloc 512")
    x_recon = residual_block_recon(x_recon, 256, 3, 2)
    print(x_recon.shape)
    x_recon = residual_block_recon(x_recon, 128, 3, 2)
    print(x_recon.shape)
    x_recon = residual_block_recon(x_recon, 64, 3, 2)
    print(x_recon.shape)
    x_recon = residual_block_recon(x_recon, 32, 3, 2)
    print(x_recon.shape)
    x_recon = Conv2DTranspose(16, 3, strides=2, padding='same', use_bias=False)(x_recon)
    x_recon = BatchNormalization()(x_recon)
    x_recon = Activation('relu')(x_recon)
    print(x_recon.shape, "unverse max pool")
    x_recon = Conv2DTranspose(16, 5, strides=2, padding='same', use_bias=False)(x_recon)
    x_recon = BatchNormalization()(x_recon)
    x_recon = Activation('relu')(x_recon)
    print(x_recon.shape)
    x_recon = Conv2DTranspose(1, 1, activation='sigmoid', padding='same', use_bias=False)(x_recon)
    print(x_recon.shape, 'recon_image')
    x_recon = tf.keras.layers.Resizing(height=256, width=256, name='recon_image')(x_recon)

    model = Model(inputs=input_img, outputs=x_recon)

    lr_schedule = ExponentialDecay(initial_learning_rate=0.001, decay_steps=10000, decay_rate=0.9, staircase=True)

    optimizer = SGD(learning_rate=lr_schedule, momentum=0.9)
    model.compile(optimizer=optimizer, loss=MeanAbsoluteError())
    return model

model = create_autoencoder()
model.summary()


In [ ]:
autoencoder = create_autoencoder()
autoencoder.summary()

In [ ]:
# Définir un callback personnalisé pour afficher la reconstruction toutes les 10 époques
class CustomCallback(tf.keras.callbacks.Callback):
    def __init__(self, model, test_data, display_loss_interval=10, display_recon_interval=50):
        super().__init__()
        self.model = model
        self.test_data = test_data
        self.display_loss_interval = display_loss_interval
        self.display_recon_interval = display_recon_interval

    def on_epoch_end(self, epoch, logs):
        if epoch % self.display_recon_interval == 0:
            reconstructions = self.model.predict(self.test_data)
            self.display_reconstruction(epoch, self.test_data, reconstructions)
        if epoch % self.display_loss_interval == 0:
            print(f"Epoch {epoch}, Loss: {logs['loss']:.6g}")


    def display_reconstruction(self, epoch, originals, reconstructions):
        n = 10  # Number of images to display
        plt.figure(figsize=(20, 4))
        for i in range(n):
            # Display original
            ax = plt.subplot(2, n, i + 1)
            plt.imshow(originals[i].reshape(256, 256), cmap='gray')
            plt.title("Original")
            plt.axis('off')
            
            # Display reconstruction
            ax = plt.subplot(2, n, i + 1 + n)
            plt.imshow(reconstructions[i].reshape(256, 256), cmap='gray')
            plt.title("Reconstructed")
            plt.axis('off')
        plt.suptitle(f'Epoch {epoch}')
        plt.show()

ethnie = 'afro_americans'
x_train = ethnies[ethnie][2] ; y_train = ethnies[ethnie][3]
x_val   = ethnies[ethnie][4] ; y_val   = ethnies[ethnie][5]
x_print = x_val.copy(); np.random.shuffle(x_print); x_print = x_print[:10]

# Entraîner le modèle avec le callback personnalisé
model.fit(x_train, x_train, epochs=5000, batch_size=32, callbacks=[CustomCallback(model, x_print)], verbose = 0)


In [ ]:
dico_latent = pca(dico_latent, "afro_americans")

In [ ]:
def create_autoencoder(shape=(256, 256, 1), input_latent=512):
    input_img = Input(shape=shape, name='input_image')

    # Encoder
    x = Conv2D(16, 5, strides=2, padding='same', use_bias=False)(input_img)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(2, strides=2, padding='same')(x)
    x = residual_block(x, 32, 3, 2)
    x = residual_block(x, 64, 3, 2)
    x = residual_block(x, 128, 3, 2)
    x = residual_block(x, 256, 3, 2)
    x = residual_block(x, 512, 3, 2)
    x = Conv2D(input_latent, 3, strides=2, padding='same', use_bias=False)(x)

    # Latent space
    latent_space_layer = Dense(input_latent, activation='relu', use_bias=False)(x)
    latent_space_layer_norm = BatchNormalization(name='latent_space_layer_norm')(latent_space_layer)

    # Decoder
    reshape_layer = Reshape(target_shape=(1, 1, input_latent))(latent_space_layer_norm)
    x_recon = residual_block_recon(reshape_layer, input_latent, 3, 2)
    x_recon = residual_block_recon(x_recon, 256, 3, 2)
    x_recon = residual_block_recon(x_recon, 128, 3, 2)
    x_recon = residual_block_recon(x_recon, 64, 3, 2)
    x_recon = residual_block_recon(x_recon, 32, 3, 2)
    x_recon = Conv2DTranspose(16, 2, strides=2, padding='same', use_bias=False)(x_recon)
    x_recon = BatchNormalization()(x_recon)
    x_recon = Activation('relu')(x_recon)
    x_recon = Conv2DTranspose(16, 5, strides=2, padding='same', use_bias=False)(x_recon)
    x_recon = BatchNormalization()(x_recon)
    x_recon = Activation('relu')(x_recon)
    x_recon = Conv2DTranspose(1, 1, activation='sigmoid', padding='same', use_bias=False)(x_recon)
    x_recon = tf.keras.layers.Resizing(height=256, width=256, name='recon_image')(x_recon)

    model = Model(inputs=input_img, outputs=x_recon)

    lr_schedule = ExponentialDecay(initial_learning_rate=0.001, decay_steps=10000, decay_rate=0.9, staircase=True)
    optimizer = SGD(learning_rate=lr_schedule, momentum=0.9)
    model.compile(optimizer=optimizer, loss=MeanAbsoluteError())
    return model

ethnie = 'afro_americans'
x_train = ethnies[ethnie][2] ; y_train = ethnies[ethnie][3]
x_val   = ethnies[ethnie][4] ; y_val   = ethnies[ethnie][5]

# Check for proper data shape
print(x_train.shape)
print(x_val.shape)

# Create and compile model
model_autoencoder = create_autoencoder()

# Train the model
history = model_autoencoder.fit(x_train, x_train, batch_size=32, epochs=3000,
                                validation_data=(x_val, x_val), shuffle=True, verbose=1)


In [ ]:
model_autoencoder = create_autoencoder()
model_autoencoder.summary()

In [ ]:
ethnie = 'afro_americans'
x_train = ethnies[ethnie][2] ; y_train = ethnies[ethnie][3]
x_val   = ethnies[ethnie][4] ; y_val   = ethnies[ethnie][5]

In [ ]:
# Entraînement du modèle
history = model_autoencoder.fit(x_train, x_train, batch_size=32, epochs=200,
                    validation_data=(x_val, y_val), shuffle=True,
                    verbose=1)

In [18]:
def create_classifier(shape=(256, 256, 1), input_latent=512, len_vecteur=56):
    nbr_filters = [32, 64, 128, 256, 512, 1024, 2048]
    input_img = Input(shape=shape, name='input_image')
    x = Conv2D(16, 3, strides=2, padding='same', use_bias=False)(input_img)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    #encoder
    for filtres in nbr_filters:
        x = residual_block(x, filtres, 3, 2)
        print(x.shape)
    x = residual_block(x, input_latent, 3, 1)
    print(x.shape)
    x = Flatten()(x)

    # Espace latent
    latent_space_layer = Dense(input_latent, activation='relu', use_bias=False)(x)
    latent_space_layer_norm = BatchNormalization(name='latent_space_layer_norm')(latent_space_layer)

    # Classification
    x_class = Dense(50, activation='relu', kernel_regularizer=l2(0.01))(latent_space_layer_norm)
    x_class = BatchNormalization()(x_class)
    x_class = Dense(25, activation='relu', kernel_regularizer=l2(0.01))(x_class)
    x_class = BatchNormalization()(x_class)
    x_class = Dense(len_vecteur, activation='softmax', name='classif')(x_class)

    model = Model(inputs=input_img, outputs=x_class)
    
    lr_schedule = ExponentialDecay(initial_learning_rate=0.001, decay_steps=10000, decay_rate=0.9, staircase=True)

    optimizer = SGD(learning_rate=lr_schedule, momentum=0.9)
    model.compile(optimizer=optimizer, loss=CategoricalCrossentropy(), metrics=CategoricalAccuracy())
    return model

In [24]:
model = create_classifier(len_vecteur=22)
model.summary()

(None, 64, 64, 32)
(None, 32, 32, 64)
(None, 16, 16, 128)
(None, 8, 8, 256)
(None, 4, 4, 512)
(None, 2, 2, 1024)
(None, 1, 1, 2048)
(None, 1, 1, 512)
Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_image (InputLayer)       [(None, 256, 256, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_138 (Conv2D)            (None, 128, 128, 16  144         ['input_image[0][0]']            
                                )                                                                 
                                                                                                  
 batch_normalization_150 (Batch  (None, 1

In [25]:
ethnie = 'afro_americans'
x_train = ethnies[ethnie][2] ; y_train = ethnies[ethnie][3]
x_val   = ethnies[ethnie][4] ; y_val   = ethnies[ethnie][5]

In [26]:
# Entraînement du modèle
history = model.fit(x_train, y_train, batch_size=32, epochs=25,
                    validation_data=(x_val, y_val), shuffle=True,
                    verbose=1)

Epoch 1/25
137/137 [==============================] - 22s 143ms/step - loss: 4.4345 - categorical_accuracy: 0.0988 - val_loss: 4.2805 - val_categorical_accuracy: 0.0875
Epoch 2/25
137/137 [==============================] - 18s 129ms/step - loss: 3.8836 - categorical_accuracy: 0.1859 - val_loss: 3.8971 - val_categorical_accuracy: 0.1500
Epoch 3/25
137/137 [==============================] - 18s 129ms/step - loss: 3.5751 - categorical_accuracy: 0.2613 - val_loss: 3.2623 - val_categorical_accuracy: 0.3000
Epoch 4/25
137/137 [==============================] - 18s 130ms/step - loss: 3.2607 - categorical_accuracy: 0.3417 - val_loss: 3.0160 - val_categorical_accuracy: 0.4000
Epoch 5/25
137/137 [==============================] - 18s 130ms/step - loss: 2.9655 - categorical_accuracy: 0.4247 - val_loss: 2.8680 - val_categorical_accuracy: 0.4875
Epoch 6/25
137/137 [==============================] - 18s 130ms/step - loss: 2.7214 - categorical_accuracy: 0.4873 - val_loss: 2.6857 - val_categorical_acc

In [ ]:
history = model.fit(x_train, y_train, batch_size=32, epochs=100, 
                    validation_data=(x_val, y_val), shuffle=True, verbose=1)

In [ ]:
ethnie = 'afro_americans'
x_train = ethnies[ethnie][2] ; y_train = ethnies[ethnie][3]
x_val   = ethnies[ethnie][4] ; y_val   = ethnies[ethnie][5]

# Création du modèle
model = create_hybride()

In [ ]:
# Entraînement du modèle
history = model.fit(x_train, {'recon_image': x_train, 'classif': y_train},
                    validation_data=(x_val, {'recon_image': x_val, 'classif': y_val}),
                    epochs=1000, batch_size=32, verbose=1)

In [ ]:
predictions = model_autoencoder.predict(x_val)
x_val_predict = predictions

In [ ]:
get_latent = tf.keras.Model(inputs=model_autoencoder.input, outputs=model_autoencoder.get_layer('latent_space_layer_norm').output)

In [ ]:
get_latent.output_shape

In [ ]:
afro_americans_latent = get_latent.predict(x_val).reshape(x_val.shape[0], 512)
caucasian_latent = get_latent.predict(ethnies['caucasians'][-2]).reshape(ethnies['caucasians'][-2].shape[0], 512)
asians_latent = get_latent.predict(ethnies['asians'][-2]).reshape(ethnies['asians'][-2].shape[0], 512)

In [ ]:
afro_americans_latent.shape, caucasian_latent.shape, asians_latent.shape

In [ ]:
x_val_predict.shape

In [ ]:
# Définir le nombre de lignes et de colonnes
rows = len(x_val)  # nombre d'images à afficher
cols = 2  # 2 colonnes, une pour l'image et une pour sa prédiction

fig, axes = plt.subplots(30, cols, figsize=(10, 80))

# Afficher les images dans la grille avec des légendes
for i in range(30):
    # Afficher l'image de x_val
    ax_val = axes[i, 0]
    ax_val.imshow(x_val[i])
    ax_val.set_title(f'image[{i}]')
    ax_val.axis('off')
    
    # Afficher l'image de x_train
    ax_train = axes[i, 1]
    ax_train.imshow(x_val_predict[i])
    ax_train.set_title(f'reconstruction[{i}]')
    ax_train.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
y_val_predict[1]

In [ ]:
from scipy.spatial import distance

rows = len(x_val)

cpt_correct = 0
for i in range(rows):
    # distance pred à ground truth
    correct_vector_distance = distance.euclidean(y_val[i], y_val_predict[i])
    # distances entre la pred et tous les autres vecteurs
    other_distances = [distance.euclidean(y_val[i], y_val[j]) for j in range(rows) if j != i]
    #vérifie si la distance est bien la plus petite de toutes
    is_correct_prediction = correct_vector_distance <= min(other_distances)
    if is_correct_prediction:
        cpt_correct += 1

cpt_correct

In [ ]:
from sklearn.decomposition import PCA


def pca(dico_latent, ethnie):
    scatters = []
    pca = PCA(n_components=2)
    plt.figure(figsize=(10, 8))
    for latent in dico_latent.keys():
        latent_pca = pca.fit_transform(dico_latent[latent][0])
        dico_latent[latent].append(latent_pca)  # ajout de latent_pca
        scatter = plt.scatter(latent_pca[:, 0], latent_pca[:, 1], alpha=0.5, label=latent)
        scatters.append(scatter)
    plt.legend(handles=scatters)
    plt.title(f'Projection ACP de l espace latent {ethnie}')
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.show()

    return dico_latent

In [ ]:
dico_latent = {'afro_americans': [afro_americans_latent], 
               'caucasians': [caucasian_latent], 
               'asians': [asians_latent]}

In [ ]:
dico_latent = pca(dico_latent, "afro_americans")

In [ ]:
def calculate_mean_dispersion(latent_representations):
    centroid = np.mean(latent_representations, axis=0)
    distance_squared = np.sum((latent_representations - centroid)**2, axis = 1)
    return np.mean(distance_squared)

def comparaison_dispersion(dico_latent):
    for ethnie in dico_latent.keys():
        dispersion = calculate_mean_dispersion(dico_latent[ethnie][1])
        print(f"Moyenne des dispersions des espaces latents pour les individus {ethnie} : {dispersion}")

In [ ]:
comparaison_dispersion(dico_latent)